# Machine Learning and Pattern Recognition (MLPR)

# Assignment 2

# Georgios Pligoropoulos - s1687568

## November 2016 - 1st Semester

In [62]:
import math
import imp
from scipy import io
import numpy as np
from matplotlib import pyplot as plt
from sklearn.cross_validation import train_test_split

#or notebook
%matplotlib inline

In [2]:
import sys
mlprDir = '/home/student/Dropbox/MSc_Artificial_Intelligence/mlpr_Machine_Learning_Pattern_Recognition/mlpr'
sys.path.append(mlprDir)

In [6]:
randomSeed = 0

In [17]:
ctData = io.loadmat('ct_data.mat', squeeze_me=True) #“squeeze_me=True” so thaty_train.shapeis(N,)ratherthan(N,1)
ctDataLen = len(ctData)
ctDataLen

9

In [18]:
ctData.keys()

['X_val',
 'X_train',
 'X_test',
 '__header__',
 '__globals__',
 'y_val',
 'y_train',
 '__version__',
 'y_test']

In [21]:
assert len(ctData['y_train'].shape) == 1, "the squeeze me = True above has failed"

In [30]:
def convertDictionaryToVariables(dictionary):
    """http://stackoverflow.com/questions/18090672/convert-dictionary-entries-into-variables-python"""
    for key,val in dictionary.items():
        exec(key + '=val')

In [31]:
convertDictionaryToVariables(ctData) # TODO: does not work in python notebook ??

In [32]:
Xtrain, Xval, Xtest, yTrain, yVal, yTest = ctData['X_train'], ctData['X_val'], ctData['X_test'], \
    ctData['y_train'], ctData['y_val'], ctData['y_test']

In [33]:
ctData['__header__']

'MATLAB 5.0 MAT-file, written by Octave 4.0.0, 2016-10-30 15:41:35 UTC'

In [34]:
ctData['__globals__']

[]

In [35]:
ctData['__version__']

'1.0'

In [39]:
Xtrain.shape, yTrain.shape

((40754, 384), (40754,))

In [40]:
Xval.shape, yVal.shape

((5785, 384), (5785,))

In [41]:
Xtest.shape, yTest.shape

((6961, 384), (6961,))

In [55]:
someWhereInTheMiddleOfTraining = range(len(Xtrain)/2,len(Xtrain)/2 + 6)

In [56]:
Xtrain[someWhereInTheMiddleOfTraining]

array([[ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      , -0.25    ],
       [ 0.      ,  0.      ,  0.      , ...,  0.963103,  0.      , -0.25    ],
       [ 0.      ,  0.      ,  0.      , ...,  0.975469,  0.      , -0.25    ],
       [ 0.      ,  0.      ,  0.      , ...,  0.807768,  0.      , -0.25    ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      , -0.25    ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      , -0.25    ]])

In [57]:
yTrain[someWhereInTheMiddleOfTraining]

array([-0.39463686,  0.37122723,  0.37342168,  0.36903278,  0.33611602,
        0.35147717])

### 1a

Verifying that the mean of the training positions in yTrain is zero

In [60]:
np.allclose(np.mean(yTrain), 0)

True

The mean of the 5,785 positions in they_valarray is not zero.

In [65]:
yValMean = np.mean(yVal)
yValMean

-0.21600850932415991

If we gathered a second dataset and computed its mean in the same way, we would get a different mean. For some datasets the mean will be bigger than the underlying true mean, sometimes it will be smaller.  
Here we do not have a second dataset and thus we are stuck with that. But we can use the standard deviation to see how much the mean would vary.

In [70]:
def getMeanError(arr):
    return np.std(arr)/math.sqrt(len(arr))

In [73]:
validationMeanError = getMeanError(yVal)
print "The standard error of the validation mean is %f +/- %f" % (yValMean, validationMeanError)

The standard error of the validation mean is -0.216009 +/- 0.012903


In [75]:
firstPartOfTraining = yTrain[:len(yVal)]

In [76]:
firstPartOfTrainingMeanError = getMeanError(firstPartOfTraining)
print "The standard error of the first part of training mean is %f +/- %f" %\
    (np.mean(firstPartOfTraining), firstPartOfTrainingMeanError)

The standard error of the first part of training mean is -0.442477 +/- 0.011926


Why these standard error bars do not reliably indicate what the average of locations in future CT slice data will be?

Well intuitively the CT scan is a complex process and it involves the complexity of the human body. Here we have less than 60.000 instances in total which is a very small number in comparison with the population. Given that the person is totally still. Because if the person is slightly moving when the CT scan is being operating this will add noise to the data giving slight variations to the inputs thus changing the mean.
We need many more datasets to derive a better mean and have a better approximation of the standard error.  
In addition we have taken into account the **assumption that the observations are independent** from each other which might not be the case.

### 1b

In [103]:
def checkIfArrayContainsIdenticalElements(arr):
    assert len(arr) >= 1, "do not call this function if array is empty"
    #reduce(lambda x,y : x if x == y else False, arr) == arr[0]
    return np.all(np.array(arr) == arr[0])

In [104]:
def identifyRedundantAttributes(dataset):
    """returns a binary mask suggesting the columns where all the instances have the same value
    thus contributing zero information"""
    return np.apply_along_axis(checkIfArrayContainsIdenticalElements, axis=0, arr = dataset)

In [110]:
redundantAttrs = identifyRedundantAttributes(Xtrain)
redundantAttrs.shape

(384,)

In [120]:
Xtrain.shape

(40754, 384)

In [125]:
def removeRedundantAttrs(dataset, redundantAttributes):
    fixedDataset = np.delete(dataset, np.argwhere(redundantAttributes), axis=1)
    assert len(redundantAttributes[redundantAttributes==True]) == dataset.shape[1] - fixedDataset.shape[1],\
        "%d vs %d" % (len(redundantAttributes[redundantAttributes==True]), dataset.shape[1] - fixedDataset.shape[1])
    return fixedDataset

In [126]:
Xtr = removeRedundantAttrs(Xtrain, redundantAttrs)
Xtr.shape

(40754, 379)

In [127]:
Xvalid = removeRedundantAttrs(Xval, redundantAttrs)
Xvalid.shape

(5785, 379)

In [128]:
Xtesting = removeRedundantAttrs(Xtest, redundantAttrs)
Xtesting.shape

(6961, 379)

We have removed the following columns:

In [129]:
np.argwhere(redundantAttrs)

array([[ 59],
       [ 69],
       [179],
       [189],
       [351]])